In [1]:
import pandas as pd
import time
from os.path import join
import json 
import numpy as np 
from tqdm import tqdm
import seaborn as sns
from os import listdir, path
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import missingno as msno
from matplotlib.colors import ListedColormap
import plotly.graph_objs as go
# make a train test split 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
# Importar librerías necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
df = pd.read_parquet('../data/preprocessed_data/training_data/df_failures_telemetry.parquet')



In [6]:
df.value_counts('failure')

failure
0    654052
2       187
1       153
4       138
3       103
dtype: int64

In [10]:
df.query('failure != 0')#.value_counts('model')


,datetime,machineID,volt,rotate,pressure,vibration,failure,failure_binary
96,2015-01-05 06:00:00,1,179.303153,499.777962,111.833028,52.383097,4,1
1536,2015-03-06 06:00:00,1,198.257975,456.862342,89.333995,38.671900,1,1
2616,2015-04-20 06:00:00,1,180.050801,346.362480,105.661164,39.218055,2,1
4056,2015-06-19 06:00:00,1,187.673963,493.005160,105.334392,53.963961,4,1
5856,2015-09-02 06:00:00,1,144.094532,409.380150,106.720871,57.454990,4,1
...,...,...,...,...,...,...,...,...
646269,2015-07-17 06:00:00,99,173.273231,490.753609,135.766664,37.678738,3,1
646629,2015-08-01 06:00:00,99,147.542660,471.754736,104.324517,42.983190,2,1
647349,2015-08-31 06:00:00,99,189.337240,450.150198,103.996522,40.219528,3,1
649095,2015-02-12 06:00:00,100,226.666914,484.516473,97.521897,53.129437,1,1


In [ ]:
df.iloc[1500:1540]


In [109]:
ventana_features = 40
ventana_falla = 10

def seleccionar_ventanas(ventana_features,ventana_falla):

    df_ventana = pd.DataFrame( )
    for id_falla,i_falla in enumerate(df.query('failure != 0').index):
        shift = np.random.randint(1,ventana_falla+1)
        df_temp = df.iloc[i_falla -ventana_features + shift: i_falla + shift].copy()
        df_temp = df_temp.iloc[0:ventana_features-ventana_falla]
        df_temp['id_falla'] = id_falla
        df_temp.iloc[ventana_features-ventana_falla-1,6] = df.iloc[i_falla,6]
        df_temp.iloc[ventana_features-ventana_falla-1,7] = df.iloc[i_falla,7]
        df_ventana = pd.concat([df_ventana,df_temp]) 
    return df_ventana


df_ventana = seleccionar_ventanas(ventana_features,ventana_falla)


In [108]:
df_ventana.value_counts('failure')

failure
0    6391
2     187
1     153
4     138
3     103
dtype: int64

In [111]:
df_ventana

,datetime,machineID,volt,rotate,pressure,vibration,failure,failure_binary,id_falla
65,2015-01-03 23:00:00,1,179.369497,374.127148,87.737435,49.153972,0,0,0
66,2015-01-04 00:00:00,1,195.173086,436.173620,103.833238,49.947102,0,0,0
67,2015-01-04 01:00:00,1,168.969595,467.285905,117.148227,58.392050,0,0,0
68,2015-01-04 02:00:00,1,165.187401,475.126937,114.584421,57.549016,0,0,0
69,2015-01-04 03:00:00,1,192.065086,506.595129,113.672301,57.809162,0,0,0
...,...,...,...,...,...,...,...,...,...
654122,2015-09-09 17:00:00,100,190.661450,427.957583,106.073603,43.027098,0,0,580
654123,2015-09-09 18:00:00,100,185.285090,530.170833,90.284502,37.529162,0,0,580
654124,2015-09-09 19:00:00,100,183.177725,399.576474,100.839263,43.338658,0,0,580
654125,2015-09-09 20:00:00,100,202.003791,432.215269,113.254253,40.246474,0,0,580


In [128]:

df_ventana2 = pd.DataFrame( )

for ix,id  in enumerate( range(1000,len(df)-1000,800)):
    df_temp = df.iloc[ix:ix+ventana_features].copy()
    if df_temp['failure'].max()!=0:
        continue
    df_temp['id_falla'] = ix
    df_ventana2 = pd.concat([df_ventana2,df_temp])
    

776

AttributeError: 'list' object has no attribute 'unique'

,datetime,machineID,volt,rotate,pressure,vibration,failure,failure_binary,id_falla
5816,2015-08-31 14:00:00,1.0,154.549723,432.808321,98.728875,55.246831,0.0,0.0,4.0
5817,2015-08-31 15:00:00,1.0,143.911321,482.729506,111.432430,43.475987,0.0,0.0,4.0
5818,2015-08-31 16:00:00,1.0,171.687714,394.202261,90.972853,42.354253,1.0,0.0,4.0
5819,2015-08-31 17:00:00,1.0,164.648960,448.719026,97.926104,53.446291,0.0,0.0,4.0
5820,2015-08-31 18:00:00,1.0,172.996607,421.424571,79.267492,40.510669,0.0,0.0,4.0
5821,2015-08-31 19:00:00,1.0,150.968899,484.243200,110.642836,58.889644,0.0,0.0,4.0
5822,2015-08-31 20:00:00,1.0,180.248624,406.823174,96.406769,56.275566,0.0,0.0,4.0
5823,2015-08-31 21:00:00,1.0,174.102013,481.779100,101.953891,50.037437,0.0,0.0,4.0
5824,2015-08-31 22:00:00,1.0,162.250917,549.741973,118.961608,43.489968,0.0,0.0,4.0
5825,2015-08-31 23:00:00,1.0,168.963619,454.954788,95.816718,57.328544,0.0,0.0,4.0
